In [11]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re

train_data = pd.read_csv('/code/titanic/train.csv')
test_data = pd.read_csv('/code/titanic/test.csv')

In [12]:
for data in [train_data, test_data]:
    data['Name'] = data['Name'].apply(lambda x: re.sub('^[^,]+, ', '', x))
    data['Name'] = data['Name'].apply(lambda x: re.sub(' .+', '', x))

    data['Name'][data['Name'] == 'Mlle.'] = 'Miss.'
    data['Name'][data['Name'] == 'Mme.'] = 'Mrs.'
    data['Name'][data['Name'] == 'Ms.'] = 'Miss.'
    data['Name'][data['Name'] == 'Lady.'] = 'Miss.'
    
    data['Name'][data['Name'] == 'Mrs.'] = 'Miss.'

    # data['Name'][data['Name'] == 'Master.'] = 'Mr.'
    data['Name'][data['Name'] == 'Sir.'] = 'Mr.'
    data['Name'][data['Name'] == 'the'] = 'Mr.'

    data['Name'][data['Name'] == 'Dr.'] = 'Staff.'
    data['Name'][data['Name'] == 'Capt.'] = 'Staff.'

    data['Name'][data['Name'] == 'Don.'] = 'Noble.'
    data['Name'][data['Name'] == 'Dona.'] = 'Noble.'
    data['Name'][data['Name'] == 'Rev.'] = 'Noble.'
    data['Name'][data['Name'] == 'Jonkheer.'] = 'Noble.'

    data['Name'][data['Name'] == 'Major.'] = 'Military.'
    data['Name'][data['Name'] == 'Col.'] = 'Military.'

/tmp/ipykernel_9740/463859450.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name'][data['Name'] == 'Mlle.'] = 'Miss.'
/tmp/ipykernel_9740/463859450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name'][data['Name'] == 'Mme.'] = 'Mrs.'
/tmp/ipykernel_9740/463859450.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name'][data['Name'] == 'Ms.'] = 'Miss.'
/tmp/ipykernel_9740/463859450.py:8: SettingWithCopyWa

In [13]:
for data in [train_data, test_data]:
    data['Family'] = data['SibSp'] + data['Parch'] + 1
    # print(data['Family'].value_counts())
    data.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [14]:
for data in [train_data, test_data]:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
    data['Fare'] = data['Fare'] / data['Family']

In [15]:
train_data.drop(['PassengerId'], axis=1, inplace=True)
for data in [train_data, test_data]:
    data.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

    data['Age'].fillna(data['Age'].median(), inplace=True)
    data.dropna(inplace=True)


    data['Age'] = data['Age'].astype('int64')

In [16]:
train_data = pd.get_dummies(train_data, columns=['Name','Pclass','Sex','Embarked'])
test_data = pd.get_dummies(test_data, columns=['Name','Pclass','Sex','Embarked'])

In [17]:
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

x = train_data.drop(['Survived'], axis=1)
y = train_data['Survived']


SPLIT_STATE = 5
PARAMS = {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 12, 'n_jobs': None, 'oob_score': False, 'random_state': 41, 'verbose': 0, 'warm_start': False}


train_x, check_x, train_y, check_y = train_test_split(x, y, random_state=SPLIT_STATE, test_size=0.2)

model = RandomForestClassifier(**PARAMS)
model.fit(train_x, train_y)
print(accuracy_score(check_y, model.predict(check_x)))

0.8764044943820225


In [18]:
test_data

,PassengerId,Age,Fare,Family,Name_Master.,Name_Military.,Name_Miss.,Name_Mr.,Name_Noble.,Name_Staff.,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,34,7.829200,1,False,False,False,True,False,False,False,False,True,False,True,False,True,False
1,893,47,3.500000,2,False,False,True,False,False,False,False,False,True,True,False,False,False,True
2,894,62,9.687500,1,False,False,False,True,False,False,False,True,False,False,True,False,True,False
3,895,27,8.662500,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
4,896,22,4.095833,3,False,False,True,False,False,False,False,False,True,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,27,8.050000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
414,1306,39,108.900000,1,False,False,False,False,True,False,True,False,False,True,False,True,False,False
415,1307,38,7.250000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
416,1308,27,8.050000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True


In [19]:
submit = pd.DataFrame()

submit['PassengerId'] = test_data['PassengerId']
submit['Survived'] = model.predict(test_data.drop(['PassengerId'], axis=1))

submit.to_csv('/code/titanic/submit.csv', index=False)

In [20]:
test_data

,PassengerId,Age,Fare,Family,Name_Master.,Name_Military.,Name_Miss.,Name_Mr.,Name_Noble.,Name_Staff.,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,34,7.829200,1,False,False,False,True,False,False,False,False,True,False,True,False,True,False
1,893,47,3.500000,2,False,False,True,False,False,False,False,False,True,True,False,False,False,True
2,894,62,9.687500,1,False,False,False,True,False,False,False,True,False,False,True,False,True,False
3,895,27,8.662500,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
4,896,22,4.095833,3,False,False,True,False,False,False,False,False,True,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,27,8.050000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
414,1306,39,108.900000,1,False,False,False,False,True,False,True,False,False,True,False,True,False,False
415,1307,38,7.250000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
416,1308,27,8.050000,1,False,False,False,True,False,False,False,False,True,False,True,False,False,True
